# Flaw Engine: Cloud Trainer (Path-Fixed Edition) 🚀

### 🏁 Step 1: Clone Repository
Run this to get the code onto the machine.

In [ ]:
GITHUB_URL = "https://github.com/Gideon-gg101/flaw-engine.git"

import os
import shutil

print("Cleaning environment...")
!rm -rf /content/flaw_repo

print(f"Cloning {GITHUB_URL}...")
!git clone {GITHUB_URL} /content/flaw_repo

if os.path.exists('/content/flaw_repo'):
    print("\n✅ Clone Successful!")
    # Locate the actual project root (some repos have a nested folder)
    %cd /content/flaw_repo
else:
    print("\n❌ FAILED to clone.")

### 🏁 Step 2: Build C++ Core
This cell handles the compilation.

In [ ]:
print("Installing tools...")
!apt-get install -y cmake g++ > /dev/null
!pip install requests torch > /dev/null

import os
import shutil
import glob

REPO_ROOT = "/content/flaw_repo"
CORE_DIR = os.path.join(REPO_ROOT, "core")
BUILD_DIR = os.path.join(CORE_DIR, "build")

if not os.path.exists(CORE_DIR):
    # Detect if repo is nested (e.g., flaw_repo/flaw/core)
    possible_root = os.path.join(REPO_ROOT, "flaw")
    if os.path.exists(os.path.join(possible_root, "core")):
        REPO_ROOT = possible_root
        CORE_DIR = os.path.join(REPO_ROOT, "core")
        BUILD_DIR = os.path.join(CORE_DIR, "build")

print(f"Using project root: {REPO_ROOT}")
os.makedirs(BUILD_DIR, exist_ok=True)

# 1. Compile
%cd {BUILD_DIR}
!cmake ..
!make -j$(nproc)

# 2. Deploy
print("Deploying library...")
lib_files = glob.glob("flaw_core*.so")
if not lib_files:
    print("❌ Build failed! No .so file found.")
else:
    source_lib = os.path.abspath(lib_files[0])
    # Root of project and AI folder
    shutil.copy2(source_lib, os.path.join(REPO_ROOT, "flaw_core.so"))
    shutil.copy2(source_lib, os.path.join(REPO_ROOT, "ai", "flaw_core.so"))
    print("✅ Deployed library to project.")

%cd {REPO_ROOT}
print("\n✅ READY!")

### 🏁 Step 3: Start Training
Replace YOUR_URL with your ngrok address.

In [ ]:
MASTER_URL = "https://scabbardless-haywood-spangly.ngrok-free.dev"

import sys
import os

# Use the path we discovered in Step 2
try:
    if REPO_ROOT not in sys.path: sys.path.insert(0, REPO_ROOT)
    import flaw_core
    from ai.selfplay_worker import DistributedWorker
    print("🚀 Starting worker...")
    worker = DistributedWorker(MASTER_URL, depth=1)
    worker.run()
except Exception as e:
    print(f"❌ ERROR: {e}")